## Nume studenti:
- Alexandra Manole
- Teodor Mihaescu

## Grupa: 382

# Data Set 1: Breast Cancer
### (Missing values: yes)

In [2268]:
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV

In [2128]:
# Citeste datele
data = pd.read_csv('./Datasets/breast-cancer.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Class       286 non-null    object
 1   Age         286 non-null    object
 2   Menopause   286 non-null    object
 3   TumorSize   286 non-null    object
 4   InvNodes    286 non-null    object
 5   NodeCaps    278 non-null    object
 6   DegMalig    286 non-null    int64 
 7   Breast      286 non-null    object
 8   BreastQuad  285 non-null    object
 9   Irradiat    286 non-null    object
dtypes: int64(1), object(9)
memory usage: 22.5+ KB


In [2129]:
# Prelucreaza si scaleaza datele
def scale_category(name, max_value):
    category_splitted = np.char.split(np.array(data[name], dtype=str), sep='-')
    category_splitted = np.array([x for x in category_splitted], dtype=int)
    data[name] = np.mean(category_splitted, axis=1) / max_value

In [2130]:
# Class
data['Class'] = np.where(data['Class'] == 'no-recurrence-events', 0, 1)

# Age (10-99)
scale_category('Age', 99)

# Menopause
data['Menopause'] = np.where(data['Menopause'] == 'premeno', 0.0, data['Menopause'])
data['Menopause'] = np.where(data['Menopause'] == 'lt40', 0.5, data['Menopause'])
data['Menopause'] = np.where(data['Menopause'] == 'ge40', 1.0, data['Menopause'])
data['Menopause'] = data['Menopause'].astype('float64')

# Tumor Size (0-59)
scale_category('TumorSize',59)

# Involved Nodes(0-39)
scale_category('InvNodes', 39)

# Node Caps
data['NodeCaps'] = np.where(data['NodeCaps'] == 'yes', 1, data['NodeCaps'])
data['NodeCaps'] = np.where(data['NodeCaps'] == 'no', 0, data['NodeCaps'])
# data['NodeCaps'] = data['NodeCaps'].astype('float64')

# Deg Malig
data['DegMalig'] = data['DegMalig'] / 2 - 0.5

# Breast
data['Breast'] = np.where(data['Breast'] == 'left', 0, 1)

# Breast Quad
values = np.linspace(0, 1, 5)
data['BreastQuad'] = np.where(data['BreastQuad'] == 'left_low', values[0], data['BreastQuad'])
data['BreastQuad'] = np.where(data['BreastQuad'] == 'left_up', values[1], data['BreastQuad'])
data['BreastQuad'] = np.where(data['BreastQuad'] == 'right_low', values[2], data['BreastQuad'])
data['BreastQuad'] = np.where(data['BreastQuad'] == 'right_up', values[3], data['BreastQuad'])
data['BreastQuad'] = np.where(data['BreastQuad'] == 'central', values[4], data['BreastQuad'])
# data['BreastQuad'] = data['BreastQuad'].astype('float64')

# Irradiat
data['Irradiat'] = np.where(data['Irradiat'] == 'yes', 1, 0)

# pd.set_option("display.max_rows", 20, "display.max_columns", 20)
data

,Class,Age,Menopause,TumorSize,InvNodes,NodeCaps,DegMalig,Breast,BreastQuad,Irradiat
0,0,0.348485,0.0,0.542373,0.025641,0,1.0,0,0,0
1,0,0.449495,0.0,0.372881,0.025641,0,0.5,1,0.75,0
2,0,0.449495,0.0,0.372881,0.025641,0,0.5,0,0,0
3,0,0.651515,1.0,0.288136,0.025641,0,0.5,1,0.25,0
4,0,0.449495,0.0,0.033898,0.025641,0,0.5,1,0.5,0
...,...,...,...,...,...,...,...,...,...,...
281,1,0.348485,0.0,0.542373,0.025641,0,0.5,0,0.25,0
282,1,0.348485,0.0,0.372881,0.025641,0,1.0,0,0.25,1
283,1,0.651515,1.0,0.372881,0.025641,0,0.0,1,0.25,0
284,1,0.449495,1.0,0.542373,0.102564,0,1.0,0,0,0


In [2131]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Class       286 non-null    int32  
 1   Age         286 non-null    float64
 2   Menopause   286 non-null    float64
 3   TumorSize   286 non-null    float64
 4   InvNodes    286 non-null    float64
 5   NodeCaps    278 non-null    object 
 6   DegMalig    286 non-null    float64
 7   Breast      286 non-null    int32  
 8   BreastQuad  285 non-null    object 
 9   Irradiat    286 non-null    int32  
dtypes: float64(5), int32(3), object(2)
memory usage: 19.1+ KB


In [2132]:
# Missing values imputation
"""
Observam ca avem urmatoarele missing values:
- 1 x BreastQuad
- 8 x NodeCaps

Pentru BreastQuad vom folosi un fillna dupa frecventa, ce va inlocui valoare lipsa cu cea mai itnalnita valoare, fapt care nu va influenta prea mult datele noaste. Facem acest lucru deoarece nu exista o relatie prea sigura intre localizarea cancerului mamar si celelalte atribute

Pentru NodeCaps vom folosi un IterativeImputer deoarece acest atribut are legatura cu majoritatea celorlalte atribute, acesta fiind un multivariate imputer. Desi acest lucru este mai costisitor din cauza clasificarii facute de acesta. (initial_strategy='most_frequent' deoarece avem doar yes si no, nu vrem medie, mediana sau constanta)
"""
data['BreastQuad'].fillna(data['BreastQuad'].mode()[0], inplace=True)

iterativeImputer = IterativeImputer(random_state=0, initial_strategy='most_frequent')
data_columns = data.columns
data_index = data.index
data = pd.DataFrame(iterativeImputer.fit_transform(data))
data.columns = data_columns
data.index = data_index

In [2133]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Class       286 non-null    float64
 1   Age         286 non-null    float64
 2   Menopause   286 non-null    float64
 3   TumorSize   286 non-null    float64
 4   InvNodes    286 non-null    float64
 5   NodeCaps    286 non-null    float64
 6   DegMalig    286 non-null    float64
 7   Breast      286 non-null    float64
 8   BreastQuad  286 non-null    float64
 9   Irradiat    286 non-null    float64
dtypes: float64(10)
memory usage: 22.5 KB


In [2134]:
# Separarea setului de date
X = data.loc[:, 'Age':]
y = data.loc[:, :'Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, shuffle=True)

# Convert to numpy arrays
X_train_mat = X_train.to_numpy()
X_test_mat = X_test.to_numpy()
y_train_col = y_train.to_numpy()
y_test_col = y_test.to_numpy()

In [2135]:
# Verifica daca sunt egal distribuite
print(Counter(y_train_col[:,0]))
print(Counter(y_test_col[:,0]))

Counter({0.0: 131, 1.0: 59})
Counter({0.0: 70, 1.0: 26})


## Model 1: KNeighborsClassifier

In [2136]:
# Model
"""
Principiul din spatele acestei metode este de a gasi un numar predefinit de cazuri apropiate ca distanta de noul punct care se incearca a fi prezis. Alegerea vecinilor influenteaza rezultatele modelului. Am observant ca in jurul valorii de 10 a hiperparametrului n_neighbors obtinem valori mai bune. 
Algoritmul necesita multe resurse la partea de testare, iar cea de antrenare se intampla aproape instant ("retinerea" datelor).
"""
model1 = KNeighborsClassifier(n_neighbors=10)
model1.fit(X_train_mat, y_train_col)
y_hat = model1.predict(X_test_mat)

In [2137]:
print(f'Predicted:\t{y_hat}\n')
print(f'Ground truth:\t{y_test_column}\n')
print(f'Failed:\t{sum(y_hat != y_test_column)}')

Predicted:	[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Ground truth:	[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1.
 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0.]

Failed:	25


In [2138]:
print(f'Mean Absolute Error:\t{metrics.mean_absolute_error(y_test_col, y_hat)}')
print(f'Mean Squared Error:\t{metrics.mean_squared_error(y_test_col, y_hat)}')
print(f'Accuracy:\t\t{np.round(metrics.accuracy_score(y_test_col, y_hat) * 100, 3)}%')

Mean Absolute Error:	0.2604166666666667
Mean Squared Error:	0.2604166666666667
Accuracy:		73.958%


In [2139]:
# CV Train
results_train = cross_validate(model1, X_train_mat, y_train_col, scoring=['accuracy','f1'], return_train_score=True)
results_train

{'fit_time': array([0.00199366, 0.00099897, 0.00103593, 0.00102949, 0.        ]),
 'score_time': array([0.00798011, 0.00604582, 0.00763345, 0.00495267, 0.00498629]),
 'test_accuracy': array([0.76315789, 0.71052632, 0.71052632, 0.76315789, 0.73684211]),
 'train_accuracy': array([0.78947368, 0.76315789, 0.77631579, 0.75657895, 0.77631579]),
 'test_f1': array([0.30769231, 0.26666667, 0.26666667, 0.4       , 0.375     ]),
 'train_f1': array([0.55555556, 0.45454545, 0.4516129 , 0.44776119, 0.5       ])}

In [2140]:
# CV Test
results_test = cross_validate(model1, X_test_mat, y_test_col, scoring=['accuracy','f1'], return_train_score=True)
results_test

{'fit_time': array([0.00099564, 0.00099802, 0.        , 0.0007689 , 0.00106764]),
 'score_time': array([0.00564241, 0.00503135, 0.00395799, 0.00299382, 0.00691128]),
 'test_accuracy': array([0.7       , 0.73684211, 0.73684211, 0.73684211, 0.73684211]),
 'train_accuracy': array([0.75      , 0.71428571, 0.72727273, 0.72727273, 0.74025974]),
 'test_f1': array([0., 0., 0., 0., 0.]),
 'train_f1': array([0.0952381 , 0.        , 0.        , 0.08695652, 0.09090909])}

In [2141]:
# Cautare hiperparametrii optimi - GridSearch
parameter_grid = {'n_neighbors': list(range(1, 20)), 'p': [1, 2, 3]}
grid_search = GridSearchCV(estimator = KNeighborsClassifier(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train_mat, y_train_col)
results_gscv = cross_val_score(grid_search, X_test_mat, y_test_col, scoring='accuracy')

In [2142]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.7294736842105263
{'n_neighbors': 14, 'p': 2}


## Model 2: LogisticRegression

In [2164]:
# Model
"""
Aceasta metoda folosita pentru clasificare poate lucra atat cu 2 clase, cat si cu o multitudine de clase-regresie multinomiala. Practic modelul dat de aceasta regresie determina probabilitatea ca obiectul determinat de vectorul de intrare sa apartina unei clase sau celeilalte.
Am folosit multi_class='ovr' deoarece avem o regresie logistica binomiala.
"""
model2 = LogisticRegression(solver='liblinear', multi_class='ovr', max_iter=1000)
model2.fit(X_train_mat, y_train_col)
y_hat = model2.predict(X_test_mat)

In [ ]:
print(f'Predicted:\t{y_hat}\n')
print(f'Ground truth:\t{y_test_column}\n')
print(f'Failed:\t{sum(y_hat != y_test_column)}')

In [2165]:
# CV Train
results_train = cross_validate(model2, X_train_mat, y_train_col, scoring=['accuracy','f1'], return_train_score=True)
results_train

{'fit_time': array([0.00199366, 0.00199461, 0.00099683, 0.0009973 , 0.00199962]),
 'score_time': array([0.00202894, 0.0019958 , 0.00199533, 0.00199389, 0.00198722]),
 'test_accuracy': array([0.73684211, 0.81578947, 0.63157895, 0.68421053, 0.71052632]),
 'train_accuracy': array([0.73026316, 0.69736842, 0.75      , 0.76973684, 0.76315789]),
 'test_f1': array([0.28571429, 0.58823529, 0.22222222, 0.33333333, 0.42105263]),
 'train_f1': array([0.4057971 , 0.25806452, 0.47222222, 0.53333333, 0.47058824])}

In [2166]:
# CV Test
results_test = cross_validate(model2, X_test_mat, y_test_col, scoring=['accuracy','f1'], return_train_score=True)
results_test

{'fit_time': array([0.00099564, 0.00099945, 0.        , 0.00099707, 0.0009973 ]),
 'score_time': array([0.00199652, 0.0019927 , 0.00199389, 0.00100017, 0.00103211]),
 'test_accuracy': array([0.7       , 0.78947368, 0.63157895, 0.78947368, 0.68421053]),
 'train_accuracy': array([0.78947368, 0.71428571, 0.77922078, 0.75324675, 0.75324675]),
 'test_f1': array([0. , 0.5, 0. , 0.5, 0. ]),
 'train_f1': array([0.42857143, 0.08333333, 0.4137931 , 0.24      , 0.2962963 ])}

In [2167]:
# Cautare hiperparametrii optimi - GridSearch
parameter_grid = {'max_iter': [100, 1000, 10000],
    'C' : np.linspace(0.0, 2.0, 5),
    'solver': ['saga', 'liblinear', 'lbfgs']}
grid_search = GridSearchCV(estimator = LogisticRegression(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train_mat, y_train_col)
results_gscv = cross_val_score(grid_search, X_test_mat, y_test_col, scoring='accuracy')

In [2168]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.7189473684210526
{'C': 0.5, 'max_iter': 100, 'solver': 'liblinear'}


## Model 3: DecisionTreeClassifier

In [2220]:
# Model
"""
Metoda aceasta este capabila sa faca clasificarea atat pe 2 clase, cat si pe mai multe clase. In clasificarea aceasta nodurile testeaza valoarea unui atribut anume, muchiile corespund raspunsului testului si face legatura cu urmatoarele noduri, iar frunzele arborelui prezic  rezultatul.
"""
model3 = DecisionTreeClassifier(max_depth=2)
model3.fit(X_train_mat, y_train_col)
y_hat = model3.predict(X_test_mat)

In [2221]:
print(f'Predicted:\t{y_hat}\n')
print(f'Ground truth:\t{y_test_column}\n')
print(f'Failed:\t{sum(y_hat != y_test_column)}')

Predicted:	[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Ground truth:	[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1.
 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0.]

Failed:	27


In [2222]:
# CV Train
results_train = cross_validate(model3, X_train_mat, y_train_col, scoring=['accuracy','f1'], return_train_score=True)
results_train

{'fit_time': array([0.        , 0.00099802, 0.00099778, 0.00103545, 0.00099564]),
 'score_time': array([0.0029943 , 0.0019927 , 0.00202608, 0.0009644 , 0.00199604]),
 'test_accuracy': array([0.78947368, 0.68421053, 0.73684211, 0.71052632, 0.76315789]),
 'train_accuracy': array([0.75657895, 0.75657895, 0.76973684, 0.77631579, 0.76315789]),
 'test_f1': array([0.42857143, 0.25      , 0.44444444, 0.35294118, 0.47058824]),
 'train_f1': array([0.47887324, 0.47887324, 0.47761194, 0.5       , 0.47058824])}

In [2223]:
# CV Test
results_test = cross_validate(model3, X_test_mat, y_test_col, scoring=['accuracy','f1'], return_train_score=True)
results_test

{'fit_time': array([0.00099611, 0.00099897, 0.0009973 , 0.        , 0.0010283 ]),
 'score_time': array([0.00199556, 0.00299191, 0.00199485, 0.00199938, 0.0009656 ]),
 'test_accuracy': array([0.7       , 0.78947368, 0.68421053, 0.73684211, 0.78947368]),
 'train_accuracy': array([0.78947368, 0.75324675, 0.79220779, 0.79220779, 0.76623377]),
 'test_f1': array([0.        , 0.6       , 0.4       , 0.        , 0.33333333]),
 'train_f1': array([0.33333333, 0.42424242, 0.42857143, 0.42857143, 0.25      ])}

In [2229]:
# Cautare hiperparametrii optimi - GridSearch
parameter_grid = {'max_depth': list(range(3,7)),
'min_samples_split': np.linspace(1,3,5),
'min_samples_leaf': np.linspace(0.1,0.3,5),
'max_features': ['sqrt', 'log2', None],
'criterion': ['gini', 'entropy']}
grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train_mat, y_train_col)
results_gscv = cross_val_score(grid_search, X_test_mat, y_test_col, scoring='accuracy')

In [2230]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.7189473684210526
{'criterion': 'gini', 'max_depth': 4, 'max_features': 'log2', 'min_samples_leaf': 0.1, 'min_samples_split': 1.0}


## Model 4: RandomForestClassifier

In [2263]:
# Model
"""
Aceasta metode de clasificare consta intr-o multitudine de arbori de decizie care lucreaza ca un intreg. Din fiecare arbore rezulta o clasa predictie si clasa cu cele mai multe "voturi" devine modelul de predictie.
"""
model4 = RandomForestClassifier(max_depth=2, random_state=0, ccp_alpha=0.2)
model4.fit(X_train_mat, y_train_col)
y_hat = model4.predict(X_test_mat)

In [2264]:
print(f'Predicted:\t{y_hat}\n')
print(f'Ground truth:\t{y_test_column}\n')
print(f'Failed:\t{sum(y_hat != y_test_column)}')

Predicted:	[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Ground truth:	[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1.
 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0.]

Failed:	22


In [2265]:
# CV Train
results_train = cross_validate(model4, X_train_mat, y_train_col, scoring=['accuracy','f1'], return_train_score=True)
results_train

{'fit_time': array([0.24135542, 0.20544481, 0.20146465, 0.24686146, 0.21575761]),
 'score_time': array([0.01396394, 0.01396513, 0.01097107, 0.01396108, 0.01297188]),
 'test_accuracy': array([0.71052632, 0.68421053, 0.68421053, 0.68421053, 0.68421053]),
 'train_accuracy': array([0.68421053, 0.69078947, 0.69078947, 0.69078947, 0.69078947]),
 'test_f1': array([0., 0., 0., 0., 0.]),
 'train_f1': array([0., 0., 0., 0., 0.])}

In [2266]:
# CV Test
results_test = cross_validate(model4, X_test_mat, y_test_col, scoring=['accuracy','f1'], return_train_score=True)
results_test

{'fit_time': array([0.2333796 , 0.23338175, 0.21801829, 0.18550611, 0.18848252]),
 'score_time': array([0.01289511, 0.01595402, 0.00997353, 0.01096845, 0.01596069]),
 'test_accuracy': array([0.7       , 0.73684211, 0.73684211, 0.73684211, 0.73684211]),
 'train_accuracy': array([0.73684211, 0.72727273, 0.72727273, 0.72727273, 0.72727273]),
 'test_f1': array([0., 0., 0., 0., 0.]),
 'train_f1': array([0., 0., 0., 0., 0.])}

In [2279]:
# Cautare hiperparametrii optimi - GridSearch
parameter_grid = {'max_depth': list(range(3,7)),
'min_samples_split': list(range(1,4)),
'min_samples_leaf': list(range(3,7))}
grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train_mat, y_train_col)
results_gscv = cross_val_score(grid_search, X_test_mat, y_test_col, scoring='accuracy')

In [2280]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.74
{'max_depth': 4, 'min_samples_leaf': 4, 'min_samples_split': 3}


## Model 5: SVC (Support Vector Classification)

In [2273]:
# Model
"""

"""
model5 = SVC(C=2, gamma='auto')
model5.fit(X_train_mat, y_train_col)
y_hat = model5.predict(X_test_mat)

In [2274]:
print(f'Predicted:\t{y_hat}\n')
print(f'Ground truth:\t{y_test_column}\n')
print(f'Failed:\t{sum(y_hat != y_test_column)}')

Predicted:	[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0.
 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Ground truth:	[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1.
 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0.]

Failed:	28


In [2275]:
# CV Train
results_train = cross_validate(model5, X_train_mat, y_train_col, scoring=['accuracy','f1'], return_train_score=True)
results_train

{'fit_time': array([0.00199604, 0.00199413, 0.00099182, 0.00099826, 0.00199342]),
 'score_time': array([0.00299239, 0.00299263, 0.0019865 , 0.00099826, 0.00099897]),
 'test_accuracy': array([0.73684211, 0.68421053, 0.63157895, 0.68421053, 0.76315789]),
 'train_accuracy': array([0.75657895, 0.70394737, 0.77631579, 0.76315789, 0.73684211]),
 'test_f1': array([0.16666667, 0.        , 0.125     , 0.33333333, 0.4       ]),
 'train_f1': array([0.47887324, 0.08163265, 0.4516129 , 0.48571429, 0.28571429])}

In [2276]:
# CV Test
results_test = cross_validate(model5, X_test_mat, y_test_col, scoring=['accuracy','f1'], return_train_score=True)
results_test

{'fit_time': array([0.00199795, 0.00099802, 0.        , 0.00099778, 0.00099611]),
 'score_time': array([0.00199127, 0.00199342, 0.00099802, 0.00101018, 0.00302863]),
 'test_accuracy': array([0.7       , 0.73684211, 0.68421053, 0.73684211, 0.73684211]),
 'train_accuracy': array([0.76315789, 0.72727273, 0.77922078, 0.72727273, 0.74025974]),
 'test_f1': array([0. , 0. , 0.4, 0. , 0. ]),
 'train_f1': array([0.18181818, 0.        , 0.37037037, 0.        , 0.09090909])}

In [2277]:
# Cautare hiperparametrii optimi - GridSearch
parameter_grid = {'C': np.linspace(0,1,5),
'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
'gamma': ['scale', 'auto']}
grid_search = GridSearchCV(estimator = SVC(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train_mat, y_train_col)
results_gscv = cross_val_score(grid_search, X_test_mat, y_test_col, scoring='accuracy')

In [2278]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.7189473684210526
{'C': 0.5, 'degree': 1, 'gamma': 'scale', 'kernel': 'rbf'}
